In [ ]:
!pip install nibabel pillow opencv-python

In [ ]:
#This script processes medical images in grayscale, ensuring they are resized or padded to a fixed dimension of 512×512 pixels. The workflow is as follows:

#1. Define Input Path & Target Size
#root_folder_path: Directory containing subfolders with images.
#target_size: Fixed output size (512, 512).
#2. Traverse Subdirectories
#Iterates through all subdirectories within root_folder_path.
#Only processes folders with "-1" in their name.
#3. Process Image Files
#Opens images (.jpg or .png) in grayscale mode ('L').
#4. Resize or Pad Images
#If the image is larger than 512×512, it is center-cropped.
#If the image is smaller, it is padded with black (0) to maintain aspect ratio while centering the content.
#5. Save Processed Images
#Overwrites the original image with the resized/padded version.
#Prints log messages to track processing.

In [ ]:
import os
from PIL import Image, ImageOps

# 定义根文件夹路径
root_folder_path = r"path"

# 目标尺寸
target_size = (512, 512)

# 遍历所有子文件夹
for root, dirs, files in os.walk(root_folder_path):
    if "-1" in os.path.basename(root):  # 只处理文件夹名包含 "-1" 的文件夹
        print(f"Processing folder: {root}")
        
        image_files = [f for f in files if f.endswith('.jpg') or f.endswith('.png')]

        for image_file in image_files:
            image_path = os.path.join(root, image_file)
            image = Image.open(image_path).convert('L')  # 转灰度

            if image.size != target_size:
                print(f"Resizing {image_file} from {image.size} to {target_size}")

                if image.size[0] > target_size[0] or image.size[1] > target_size[1]:
                    # 居中裁剪
                    left = (image.size[0] - target_size[0]) // 2
                    top = (image.size[1] - target_size[1]) // 2
                    right = left + target_size[0]
                    bottom = top + target_size[1]
                    image = image.crop((left, top, right, bottom))
                else:
                    # 修正填充，确保最终尺寸是 (512, 512)
                    padding_x = (target_size[0] - image.size[0]) // 2
                    padding_y = (target_size[1] - image.size[1]) // 2
                    extra_x = (target_size[0] - image.size[0]) % 2
                    extra_y = (target_size[1] - image.size[1]) % 2
                    image = ImageOps.expand(image, (padding_x, padding_y, padding_x + extra_x, padding_y + extra_y), fill=0)

            image.save(image_path)  # 覆盖原图
            print(f"Saved {image_file} -> {image.size}")


In [ ]:
#This script converts a series of 2D grayscale medical images and corresponding masks into 3D NIfTI (.nii.gz) volumes for use in medical imaging analysis.

#Workflow:
#Define Input Paths:

#Reads images from 222-1 (image folder) and masks from 222-3 (mask folder).
#Assumes images and masks share identical filenames.
#Sort & Validate:

#Sorts filenames to ensure images and masks are aligned.
#Checks if the number of images and masks match.
#Load & Preprocess Data:

#Converts images and masks to grayscale ('L').
#If sizes mismatch, resizes the mask to match the corresponding image.
#Converts each image and mask into a NumPy array.
#Stack into 3D Volumes:

#Stacks 2D slices along the last axis (axis=-1), creating a 3D image stack and a 3D mask stack.
#Convert to NIfTI Format:

#Uses nibabel to create NIfTI images from NumPy arrays.
#Assigns an identity matrix (np.eye(4)) as the affine transformation.
#Save as NIfTI Files:

#Saves the resulting 3D image and mask as compressed .nii.gz files.

In [ ]:
import numpy as np
from PIL import Image
import os
import nibabel as nib

# 定义图片和mask文件夹路径
image_folder_path = "path"
mask_folder_path = "path"

# 获取该文件夹下的所有图片和mask文件名（假设文件名一致）
image_files = [f for f in os.listdir(image_folder_path) if f.endswith('.jpg') or f.endswith('.png')]
mask_files = [f for f in os.listdir(mask_folder_path) if f.endswith('.png')]

# 按文件名排序，确保图片和mask一一对应
image_files.sort()
mask_files.sort()

# 确保图片和mask数量一致
if len(image_files) != len(mask_files):
    raise ValueError("Number of images and masks do not match!")

# 读取每个图像和mask，并转换为numpy数组，然后堆叠成3D数据
image_stack = []
mask_stack = []
for image_file, mask_file in zip(image_files, mask_files):
    # 读取图片和mask
    image_path = os.path.join(image_folder_path, image_file)
    mask_path = os.path.join(mask_folder_path, mask_file)
    
    image = Image.open(image_path).convert('L')  # 转换为灰度图
    mask = Image.open(mask_path).convert('L')    # 转换为灰度图
    
    # 如果图片和mask尺寸不一致，则调整大小，使其匹配
    if image.size != mask.size:
        print(f"Resizing mask to match image size for {image_file} and {mask_file}")
        mask = mask.resize(image.size)  # 调整mask的尺寸以匹配图片
    
    # 将图片和mask转换为numpy数组
    image_array = np.array(image)
    mask_array = np.array(mask)
    
    image_stack.append(image_array)
    mask_stack.append(mask_array)

# 将图片和mask堆叠成3D体积
image_stack_3d = np.stack(image_stack, axis=-1)  # 堆叠图片
mask_stack_3d = np.stack(mask_stack, axis=-1)    # 堆叠mask

# 打印形状信息，确保维度一致
print(f"3D image stack shape: {image_stack_3d.shape}")
print(f"3D mask stack shape: {mask_stack_3d.shape}")

# 创建NIfTI图像对象
nifti_img = nib.Nifti1Image(image_stack_3d, np.eye(4))  # 图像体积
nifti_mask = nib.Nifti1Image(mask_stack_3d, np.eye(4))  # Mask体积

# 保存NIfTI文件
nifti_image_save_path = "C:\\Users\\rosekang\\Desktop\\Data-revised\\Radiomics\\shenshui\\Image\\222\\3d_image.nii.gz"
nifti_mask_save_path = "C:\\Users\\rosekang\\Desktop\\Data-revised\\Radiomics\\shenshui\\Image\\222\\3d_mask.nii.gz"

nib.save(nifti_img, nifti_image_save_path)
nib.save(nifti_mask, nifti_mask_save_path)

print(f"3D image and mask saved as NIfTI files at:\n{nifti_image_save_path}\n{nifti_mask_save_path}")
